<a href="https://colab.research.google.com/github/venezianof/booksum/blob/main/oumi_distill_a_large_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<div class="align-center">
<a href="https://oumi.ai/"><img src="https://oumi.ai/docs/en/latest/_static/logo/header_logo.png" height="200"></a>

[![Documentation](https://img.shields.io/badge/Documentation-latest-blue.svg)](https://oumi.ai/docs/en/latest/index.html)
[![Discord](https://img.shields.io/discord/1286348126797430814?label=Discord)](https://discord.gg/oumi)
[![GitHub Repo stars](https://img.shields.io/github/stars/oumi-ai/oumi)](https://github.com/oumi-ai/oumi)
</div>

👋 Welcome to Open Universal Machine Intelligence (Oumi)!

🚀 Oumi is a fully open-source platform that streamlines the entire lifecycle of foundation models - from [data preparation](https://oumi.ai/docs/en/latest/resources/datasets/datasets.html) and [training](https://oumi.ai/docs/en/latest/user_guides/train/train.html) to [evaluation](https://oumi.ai/docs/en/latest/user_guides/evaluate/evaluate.html) and [deployment](https://oumi.ai/docs/en/latest/user_guides/launch/launch.html). Whether you're developing on a laptop, launching large scale experiments on a cluster, or deploying models in production, Oumi provides the tools and workflows you need.

🤝 Make sure to join our [Discord community](https://discord.gg/oumi) to get help, share your experiences, and contribute to the project! If you are interested in joining one of the community's open-science efforts, check out our [open collaboration](https://oumi.ai/community) page.

⭐ If you like Oumi and you would like to support it, please give it a star on [GitHub](https://github.com/oumi-ai/oumi).

In [1]:
%%writefile distillation_tutorial/train.yaml

model:
  model_name: "deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B"
  trust_remote_code: true
  torch_dtype_str: "bfloat16"
  device_map: "auto"

data:
  train:
    datasets:
      - dataset_name: "text_sft_jsonl"
        dataset_path: "./distillation_tutorial/math_train_10k.jsonl"
        split: "train"
        shuffle: True
        seed: 42
    seed: 42

training:
  output_dir: "distillation_tutorial/output/finetune"

  # For a single GPU, the following gives us a batch size of 16
  # If training with multiple GPUs, feel free to reduce gradient_accumulation_steps
  per_device_train_batch_size: 2
  gradient_accumulation_steps: 8  # Reduce this to 1 for 8xA100-80GB GPUs

  # ***NOTE***
  # We set it to 10 steps to first verify that it works
  # Comment out the line below to have it train for 1 full epoch (all the data)
  # Note: 1 full epoch will take about 13 minutes on 8xA100-80GB.
  max_steps: 10

  num_train_epochs: 1
  learning_rate: 1e-4
  warmup_ratio: 0.1
  logging_steps: 10
  save_steps: 0
  max_grad_norm: 10
  weight_decay: 0.01


  trainer_type: "TRL_SFT"
  optimizer: "adamw_torch_fused"
  enable_gradient_checkpointing: True
  gradient_checkpointing_kwargs:
    use_reentrant: False
  ddp_find_unused_parameters: False
  dataloader_num_workers: "auto"
  dataloader_prefetch_factor: 32
  empty_device_cache_steps: 1

Writing distillation_tutorial/train.yaml


FileNotFoundError: [Errno 2] No such file or directory: 'distillation_tutorial/train.yaml'

In [ ]:
!oumi train -c "distillation_tutorial/train.yaml"


   ____  _    _ __  __ _____
  / __ \| |  | |  \/  |_   _|
 | |  | | |  | | \  / | | |
 | |  | | |  | | |\/| | | |
 | |__| | |__| | |  | |_| |_
  \____/ \____/|_|  |_|_____|

⠏ Loading configuration...2026-01-15 08:06:18.257077: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1768464378.285205   21111 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1768464378.294801   21111 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1768464378.328490   21111 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1768464378.328543   21111 computation_placer.cc:

In [ ]:
%%writefile distillation_tutorial/train.yaml

model:
  model_name: "deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B"
  trust_remote_code: true
  torch_dtype_str: "bfloat16"
  device_map: "auto"

data:
  train:
    datasets:
      - dataset_name: "text_sft_jsonl"
        dataset_path: "./distillation_tutorial/math_train_10k.jsonl"
        split: "train"
        shuffle: True
        seed: 42
    seed: 42

training:
  output_dir: "distillation_tutorial/output/finetune"

  # For a single GPU, the following gives us a batch size of 16
  # If training with multiple GPUs, feel free to reduce gradient_accumulation_steps
  per_device_train_batch_size: 2
  gradient_accumulation_steps: 8  # Reduce this to 1 for 8xA100-80GB GPUs

  # ***NOTE***
  # We set it to 10 steps to first verify that it works
  # Comment out the line below to have it train for 1 full epoch (all the data)
  # Note: 1 full epoch will take about 13 minutes on 8xA100-80GB.
  max_steps: 10

  num_train_epochs: 1
  learning_rate: 1e-4
  warmup_ratio: 0.1
  logging_steps: 10
  save_steps: 0
  max_grad_norm: 10
  weight_decay: 0.01


  trainer_type: "TRL_SFT"
  optimizer: "adamw_torch_fused"
  enable_gradient_checkpointing: True
  gradient_checkpointing_kwargs:
    use_reentrant: False
  ddp_find_unused_parameters: False
  dataloader_num_workers: "auto"
  dataloader_prefetch_factor: 32
  empty_device_cache_steps: 1

Overwriting distillation_tutorial/train.yaml


In [ ]:
!oumi train -c "distillation_tutorial/train.yaml"


   ____  _    _ __  __ _____
  / __ \| |  | |  \/  |_   _|
 | |  | | |  | | \  / | | |
 | |  | | |  | | |\/| | | |
 | |__| | |__| | |  | |_| |_
  \____/ \____/|_|  |_|_____|

⠸ Loading configuration...2026-01-15 08:04:39.625376: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1768464279.682452   20659 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1768464279.707685   20659 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1768464279.782112   20659 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1768464279.782261   20659 computation_placer.cc:

In [ ]:
%%writefile distillation_tutorial/train.yaml

model:
  model_name: "deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B"
  trust_remote_code: true
  torch_dtype_str: "bfloat16"
  device_map: "auto"

data:
  train:
    datasets:
      - dataset_name: "text_sft_jsonl"
        dataset_path: "./distillation_tutorial/math_train_10k.jsonl"
        split: "train"
        shuffle: True
        seed: 42
    seed: 42

training:
  output_dir: "distillation_tutorial/output/finetune"

  # For a single GPU, the following gives us a batch size of 16
  # If training with multiple GPUs, feel free to reduce gradient_accumulation_steps
  per_device_train_batch_size: 2
  gradient_accumulation_steps: 8  # Reduce this to 1 for 8xA100-80GB GPUs

  # ***NOTE***
  # We set it to 10 steps to first verify that it works
  # Comment out the line below to have it train for 1 full epoch (all the data)
  # Note: 1 full epoch will take about 13 minutes on 8xA100-80GB.
  max_steps: 10

  num_train_epochs: 1
  learning_rate: 1e-4
  warmup_ratio: 0.1
  logging_steps: 10
  save_steps: 0
  max_grad_norm: 10
  weight_decay: 0.01


  trainer_type: "TRL_SFT"
  optimizer: "adamw_torch_fused"
  enable_gradient_checkpointing: True
  gradient_checkpointing_kwargs:
    use_reentrant: False
  ddp_find_unused_parameters: False
  dataloader_num_workers: "auto"
  dataloader_prefetch_factor: 32
  empty_device_cache_steps: 1

Writing distillation_tutorial/train.yaml


In [ ]:
!oumi train -c "distillation_tutorial/train.yaml"


   ____  _    _ __  __ _____
  / __ \| |  | |  \/  |_   _|
 | |  | | |  | | \  / | | |
 | |  | | |  | | |\/| | | |
 | |__| | |__| | |  | |_| |_
  \____/ \____/|_|  |_|_____|

⠧ Loading configuration...2026-01-15 08:27:37.823522: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1768465657.868798   26309 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1768465657.883775   26309 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1768465657.923189   26309 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1768465657.923287   26309 computation_placer.cc:

In [ ]:
import oumi
import vllm

print(f"Oumi version: {oumi.__version__}")
print(f"vLLM version: {vllm.__version__}")

# Distillation Overview

In this tutorial, we'll fine-tune a small language model (SLM) from the outputs of a large language model (LLM).

We'll use the Oumi framework to streamline the process and achieve high-quality results.

We'll cover the following topics:
1. Prerequisites
2. Data Preparation & Sanity Checks
3. Training Config Preparation
4. Launching Training
5. Monitoring Progress
6. Evaluation
7. Analyzing Results
8. Inference


# Prerequisites

## Hardware
The defaults in this tutorial are scaled down for demonstration purposes.

The true values are left to code comments within each section.

We recommend 8xA100-80GB GPUs to complete in a timely manner with adequate performance.

## Oumi Installation

First, let's install Oumi and vLLM. You can find more detailed instructions [here](https://oumi.ai/docs/en/latest/get_started/installation.html). Here, we include Oumi's GPU dependencies.


In [4]:
%pip install oumi[gpu] jsonlines

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.8/54.8 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.8/53.8 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.2/91.2 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.2/47.2 kB 4.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.1/40.1 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.6/162.6 kB 12.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of opencv-python-headless to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of hydra-core to determine which version is compatible with other requirem

## Creating our working directory
For our experiments, we'll use the following folder to save the model, training artifacts, and our working configs.

In [ ]:
from pathlib import Path

tutorial_dir = "distillation_tutorial"

Path(tutorial_dir).mkdir(parents=True, exist_ok=True)

## Setup the environment

We'll need to set the following environment variables:
- [Optional] HF_TOKEN: Your [HuggingFace](https://huggingface.co/docs/hub/en/security-tokens) token, in case you want to access a private model.
- [Optional] WANDB_API_KEY: Your [wandb](https://wandb.ai) token, in case you want to log your experiments to wandb.

# Getting Started

## Model Download

For our purposes it will be much faster if we download our models first.

We'll use the `hf_transfer` package to download.

In [ ]:
!pip install hf_transfer

In [ ]:
!HF_HUB_ENABLE_HF_TRANSFER=1 \
    hf download deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B \
    --exclude original/*

In [ ]:
!HF_HUB_ENABLE_HF_TRANSFER=1 \
    hf download deepseek-ai/DeepSeek-R1-Distill-Llama-70B \
    --exclude original/*

## Baseline Evals

Before we can improve our small model, we should measure how well it performs on a benchmark compared to the larger model.

The below code will run the MMLU PRO Math task from LM Harness.

Note that this will take some time, so we've recorded our results below for your convenience:

| Model | MMLU Pro Math Accuracy |
|-------|------------------------|
| R1 Distill 1.5B | 38.49% +- 1.32% |
| R1 Distill 70B | 61.07% +- 1.33% |

### Run Evals

In [ ]:
%%writefile $tutorial_dir/eval_small.yaml

model:
  model_name: "deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B"
  torch_dtype_str: "bfloat16"
  # shard_for_eval: True # Uncomment this line for multi-gpu setups.


tasks:
  - evaluation_backend: lm_harness
    task_name: mmlu_pro_math

output_dir: "distillation_tutorial/output/evaluation"
generation:
  batch_size: 1 # LM Harness recommends BS=1 for reproducibility.
  # batch_size: 256  # Replace with 256 for 8xA100-80GB

In [ ]:
!oumi evaluate -c "$tutorial_dir/eval_small.yaml"

^C


In [ ]:
%%writefile $tutorial_dir/eval_large.yaml

model:
  model_name: "deepseek-ai/DeepSeek-R1-Distill-Llama-70B"
  torch_dtype_str: "bfloat16"
  shard_for_eval: True


tasks:
  - evaluation_backend: lm_harness
    task_name: mmlu_pro_math

output_dir: "distillation_tutorial/output/evaluation"
generation:
  batch_size: 1 # LM Harness recommends BS=1 for reproducibility.
  # batch_size: 64  # Replace with 64 for 8xA100-80GB

Writing distillation_tutorial/eval_large.yaml


In [ ]:
!oumi evaluate -c "$tutorial_dir/eval_large.yaml"

^C


## Prepare Inference Data

Now that we've set our baseline numbers, let's prepare the training data we'll use to improve 1.5B.

Given our goal is to improve MMLU Pro Math performance, we should ideally pick data that's similar.

`meta-math/MetaMathQA` is a good choice as it avoids test set contamination while being similar.

In [ ]:
import os

import datasets
import torch

from oumi.core.configs import InferenceConfig
from oumi.core.types import Conversation, Message, Role
from oumi.inference import VLLMInferenceEngine

# This is needed for vLLM to use multiple GPUs in a notebook.
# If you're not running in a notebook, you can ignore this.
os.environ["VLLM_WORKER_MULTIPROC_METHOD"] = "spawn"

In [ ]:
dataset = datasets.load_dataset(
    "meta-math/MetaMathQA",
    revision="aa4f34d",
    split="train[:10000]",  # We'll focus only on the first 10k samples.
)

data = [sample["query"] for sample in dataset]
print(data[0])
print("num samples: ", len(data))

In [ ]:
conversations = [
    Conversation(
        messages=[
            Message(role=Role.USER, content=prompt),
        ]
    )
    for prompt in data
]
print(conversations[0])

## Run Inference

Now that our data is in the right format for collecting responses, let's go ahead and run inference.

In [ ]:
%%writefile $tutorial_dir/infer_large.yaml

model:
  model_name: "deepseek-ai/DeepSeek-R1-Distill-Llama-70B"
  torch_dtype_str: "bfloat16"
  model_max_length: 8192

generation:
  max_new_tokens: 8192

In [ ]:
%%time

# Download, and load the model in memory
# This may take a while, depending on your internet speed.
# The inference engine only needs to be loaded once and can be
# reused for multiple conversations.
config_path = f"{tutorial_dir}/infer_large.yaml"
config = InferenceConfig.from_yaml(config_path)

inference_engine = VLLMInferenceEngine(
    config.model,
    tensor_parallel_size=torch.cuda.device_count(),  # use all available GPUs
    # Enable prefix caching for vLLM.
    # This is key for performance when running prompts with a long prefix,
    # such as judging or conversations with large system prompts
    # or few-shot examples.
    enable_prefix_caching=True,
)

In [ ]:
%%time

print(f"Running inference for {len(conversations)} conversations")

generations = inference_engine.infer(
    input=conversations,
    inference_config=config,
)
print(generations[0])

## Prepare Training Data

Now that we've finished collecting responses, let's go ahead and prepare the data for training and save it.

In [ ]:
conversation_dicts = [c.to_dict() for c in generations]
print(conversation_dicts[0])

In [ ]:
import pandas as pd

dataframe = pd.DataFrame(conversation_dicts)
print(dataframe)

In [ ]:
dataframe.to_json(f"{tutorial_dir}/math_train_10k.jsonl", orient="records", lines=True)

## Run Distillation

Now that the data is ready, we can begin distilling the model. For this form of distillation, we will be fully fine-tuning the model with supervised fine-tuning.

In [ ]:
%%writefile $tutorial_dir/train.yaml

model:
  model_name: "deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B"
  trust_remote_code: true
  torch_dtype_str: "bfloat16"
  device_map: "auto"

data:
  train:
    datasets:
      - dataset_name: "text_sft_jsonl"
        dataset_path: "./distillation_tutorial/math_train_10k.jsonl"
        split: "train"
        shuffle: True
        seed: 42
    seed: 42

training:
  output_dir: "distillation_tutorial/output/finetune"

  # For a single GPU, the following gives us a batch size of 16
  # If training with multiple GPUs, feel free to reduce gradient_accumulation_steps
  per_device_train_batch_size: 2
  gradient_accumulation_steps: 8  # Reduce this to 1 for 8xA100-80GB GPUs

  # ***NOTE***
  # We set it to 10 steps to first verify that it works
  # Comment out the line below to have it train for 1 full epoch (all the data) instead.
  # Note: 1 full epoch will take about 13 minutes on 8xA100-80GB.
  max_steps: 10

  num_train_epochs: 1
  learning_rate: 1e-4
  warmup_ratio: 0.1
  logging_steps: 10
  save_steps: 0
  max_grad_norm: 10
  weight_decay: 0.01


  trainer_type: "TRL_SFT"
  optimizer: "adamw_torch_fused"
  enable_gradient_checkpointing: True
  gradient_checkpointing_kwargs:
    use_reentrant: False
  ddp_find_unused_parameters: False
  dataloader_num_workers: "auto"
  dataloader_prefetch_factor: 32
  empty_device_cache_steps: 1

### Single GPU

In [ ]:
!oumi train -c "$tutorial_dir/train.yaml"

### Multi-GPU

In [ ]:
!oumi distributed torchrun -m oumi train -c "$tutorial_dir/train.yaml"

## Evaluate

Now that we have a new distilled model, let's evaluate it on the same benchmark.

In [ ]:
%%writefile $tutorial_dir/eval_small_fft.yaml

model:
  model_name: "./distillation_tutorial/output/finetune/"
  torch_dtype_str: "bfloat16"
  shard_for_eval: True


tasks:
  - evaluation_backend: lm_harness
    task_name: mmlu_pro_math

output_dir: "distillation_tutorial/output/evaluation"
generation:
  batch_size: 1 # LM Harness recommends BS=1 for reproducibility.
  # batch_size: 256  # Replace with 256 for 8xA100-80GB

In [ ]:
!oumi evaluate -c "$tutorial_dir/eval_small_fft.yaml"

## Results

After we finetuned the model following the steps above, we achieved the following results:

| Model           | Accuracy        |
|-----------------|-----------------|
| R1 Distill 1.5B | 38.49% +- 1.32% |
| Oumi R1 Distill 1.5B | 42.41% +- 1.34% |
| R1 Distill 70B  | 61.07% +- 1.33% |

# 🧭 What's Next?

Congrats on finishing this notebook! Feel free to check out our other [notebooks](https://github.com/oumi-ai/oumi/tree/main/notebooks) in the [Oumi GitHub](https://github.com/oumi-ai/oumi), and give us a star! You can also join the Oumi community over on [Discord](https://discord.gg/oumi).

📰 Want to keep up with news from Oumi? Subscribe to our [Substack](https://blog.oumi.ai/) and [Youtube](https://www.youtube.com/@Oumi_AI)!

⚡ Interested in building custom AI in hours, not months? Apply to get [early access](https://oumi-ai.typeform.com/early-access) to the Oumi Platform, or [chat with us](https://calendly.com/d/ctcx-nps-47m/chat-with-us-get-early-access-to-the-oumi-platform) to learn more!

# Task
Generate and save the `distillation_tutorial/math_train_10k.jsonl` file by loading the "meta-math/MetaMathQA" dataset, preparing conversations, configuring the VLLM inference engine with the "deepseek-ai/DeepSeek-R1-Distill-Llama-70B" model, and performing inference to produce the conversational data.

## Carica il dataset di MetaMathQA

### Subtask:
Carica il dataset 'meta-math/MetaMathQA' per ottenere i dati di input per l'inferenza.


## Carica il dataset di MetaMathQA

### Subtask:
Carica il dataset 'meta-math/MetaMathQA' per ottenere i dati di input per l'inferenza.


**Reasoning**:
Next, I will load the 'meta-math/MetaMathQA' dataset, extract the query data, and print the first sample along with the total number of samples to verify the data loading.



In [ ]:
dataset = datasets.load_dataset(
    "meta-math/MetaMathQA",
    revision="aa4f34d",
    split="train[:10000]",  # We'll focus only on the first 10k samples.
)

data = [sample["query"] for sample in dataset]
print(data[0])
print("num samples: ", len(data))

Gracie and Joe are choosing numbers on the complex plane. Joe chooses the point $1+2i$. Gracie chooses $-1+i$. How far apart are Gracie and Joe's points?
num samples:  10000


**Reasoning**:
The previous code failed because the 'datasets' library was not imported. I need to import the 'datasets' library before using `datasets.load_dataset`.



In [ ]:
import datasets

dataset = datasets.load_dataset(
    "meta-math/MetaMathQA",
    revision="aa4f34d",
    split="train[:10000]",  # We'll focus only on the first 10k samples.
)

data = [sample["query"] for sample in dataset]
print(data[0])
print("num samples: ", len(data))

README.md: 0.00B [00:00, ?B/s]

MetaMathQA-395K.json:   0%|          | 0.00/396M [00:00<?, ?B/s]

Generating train split: 0 examples [00:00, ? examples/s]

Gracie and Joe are choosing numbers on the complex plane. Joe chooses the point $1+2i$. Gracie chooses $-1+i$. How far apart are Gracie and Joe's points?
num samples:  10000


**Reasoning**:
Now that the `data` (queries) has been successfully loaded, I need to convert it into a list of `Conversation` objects. This aligns with the overall task of preparing inference data, where each query will be treated as a user message in a conversation.



In [ ]:
conversations = [
    Conversation(
        messages=[
            Message(role=Role.USER, content=prompt),
        ]
    )
    for prompt in data
]
print(conversations[0])

conversation_id=None messages=[USER: Gracie and Joe are choosing numbers on the complex plane. Joe chooses the point $1+2i$. Gracie chooses $-1+i$. How far apart are Gracie and Joe's points?] metadata={}


**Reasoning**:
The previous code failed with a `NameError` because the `Conversation`, `Message`, and `Role` classes were not defined. These classes need to be imported from `oumi.core.types` to be used.



In [ ]:
from oumi.core.types import Conversation, Message, Role

conversations = [
    Conversation(
        messages=[
            Message(role=Role.USER, content=prompt),
        ]
    )
    for prompt in data
]
print(conversations[0])

conversation_id=None messages=[USER: Gracie and Joe are choosing numbers on the complex plane. Joe chooses the point $1+2i$. Gracie chooses $-1+i$. How far apart are Gracie and Joe's points?] metadata={}


**Reasoning**:
Now that the conversation objects are prepared, I will load the inference configuration from the YAML file and initialize the VLLM inference engine, which will be used to generate responses from the large language model.



In [ ]:
import os
from oumi.core.configs import InferenceConfig
from oumi.inference import VLLMInferenceEngine

# This is needed for vLLM to use multiple GPUs in a notebook.
os.environ["VLLM_WORKER_MULTIPROC_METHOD"] = "spawn"

config_path = f"{tutorial_dir}/infer_large.yaml"
config = InferenceConfig.from_yaml(config_path)

inference_engine = VLLMInferenceEngine(
    config.model,
    tensor_parallel_size=torch.cuda.device_count(),
    enable_prefix_caching=True,
)
print("VLLMInferenceEngine initialized successfully.")

[2026-01-15 09:02:36,814][oumi][rank0][pid:4543][MainThread][INFO]][models.py:460] Using the model's built-in chat template for model 'deepseek-ai/DeepSeek-R1-Distill-Llama-70B'.


RuntimeError: Device string must not be empty

**Reasoning**:
The previous code failed because `tutorial_dir` and `torch` were not defined in the current scope. I need to re-define `tutorial_dir` and import `torch` within this cell to make the code executable.



In [ ]:
import os
import torch
from oumi.core.configs import InferenceConfig
from oumi.inference import VLLMInferenceEngine

tutorial_dir = "distillation_tutorial"

# This is needed for vLLM to use multiple GPUs in a notebook.
os.environ["VLLM_WORKER_MULTIPROC_METHOD"] = "spawn"

config_path = f"{tutorial_dir}/infer_large.yaml"
config = InferenceConfig.from_yaml(config_path)

inference_engine = VLLMInferenceEngine(
    config.model,
    tensor_parallel_size=torch.cuda.device_count(),
    enable_prefix_caching=True,
)
print("VLLMInferenceEngine initialized successfully.")

[2026-01-15 09:02:45,956][oumi][rank0][pid:4543][MainThread][INFO]][models.py:460] Using the model's built-in chat template for model 'deepseek-ai/DeepSeek-R1-Distill-Llama-70B'.


RuntimeError: Device string must not be empty

**Reasoning**:
The `FileNotFoundError` for `distillation_tutorial/infer_large.yaml` indicates that the file was not created. I need to explicitly create this configuration file by re-executing the `%%writefile` command from a previous cell before the `VLLMInferenceEngine` can load it.



In [ ]:
%%writefile $tutorial_dir/infer_large.yaml

model:
  model_name: "deepseek-ai/DeepSeek-R1-Distill-Llama-70B"
  torch_dtype_str: "bfloat16"
  model_max_length: 8192

generation:
  max_new_tokens: 8192

In [ ]:
import torch

if torch.cuda.is_available():
    print("GPU disponibile!")
    print(f"Numero di GPU: {torch.cuda.device_count()}")
    print(f"Nome GPU: {torch.cuda.get_device_name(0)}")
else:
    print("Nessuna GPU disponibile nell'ambiente Colab. Verrà utilizzata la CPU.")

Nessuna GPU disponibile nell'ambiente Colab. Verrà utilizzata la CPU.


In [ ]:
import os
import torch
import datasets
from oumi.core.configs import InferenceConfig
from oumi.core.types import Conversation, Message, Role
from transformers import AutoTokenizer, AutoModelForCausalLM, GenerationConfig
import pandas as pd

tutorial_dir = "distillation_tutorial"

# Load the dataset (re-added)
dataset = datasets.load_dataset(
    "meta-math/MetaMathQA",
    revision="aa4f34d",
    split="train[:10000]",  # We'll focus only on the first 10k samples.
)
data = [sample["query"] for sample in dataset]

# Prepare conversations (re-added)
conversations = [
    Conversation(
        messages=[
            Message(role=Role.USER, content=prompt),
        ]
    )
    for prompt in data
]


# Define the CPU-compatible model to use for inference
# The 70B model is too large for CPU inference in this environment.
# We will use the 1.5B model as the teacher for this CPU-bound inference step.
CPU_TEACHER_MODEL_NAME = "deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B"

# Create a dummy config to hold the model and generation parameters for transformers
# This bypasses the need for a specific Oumi inference engine class
class DummyModelConfig:
    def __init__(self, model_name, torch_dtype_str, model_max_length):
        self.model_name = model_name
        self.torch_dtype_str = torch_dtype_str
        self.model_max_length = model_max_length
        self.trust_remote_code = True # DeepSeek models often require this

class DummyGenerationConfig:
    def __init__(self, max_new_tokens):
        self.max_new_tokens = max_new_tokens

dummy_config = InferenceConfig(model=DummyModelConfig(
    model_name=CPU_TEACHER_MODEL_NAME,
    torch_dtype_str="float32", # Force float32 for CPU inference
    model_max_length=8192
), generation=DummyGenerationConfig(max_new_tokens=8192))

# Load model and tokenizer directly using transformers
print(f"Loading model: {dummy_config.model.model_name} for CPU inference...")
tokenizer = AutoTokenizer.from_pretrained(dummy_config.model.model_name)
model = AutoModelForCausalLM.from_pretrained(
    dummy_config.model.model_name,
    torch_dtype=torch.float32, # Force float32 for CPU
    trust_remote_code=dummy_config.model.trust_remote_code,
).to("cpu")

# Set up generation config
generation_config = GenerationConfig.from_pretrained(dummy_config.model.model_name)
generation_config.max_new_tokens = dummy_config.generation.max_new_tokens
generation_config.pad_token_id = tokenizer.pad_token_id if tokenizer.pad_token_id is not None else tokenizer.eos_token_id
generation_config.eos_token_id = tokenizer.eos_token_id

print("Model and tokenizer loaded successfully for CPU inference.")

# --- Inference loop ---
print(f"Running inference for {len(conversations[:5])} conversations") # Adjusted print statement
generated_responses = []

for i, conv in enumerate(conversations[:5]): # Modified to limit to 5 conversations
    input_text = tokenizer.apply_chat_template(conv.to_dict()["messages"], tokenize=False, add_generation_prompt=True)
    inputs = tokenizer(input_text, return_tensors="pt").to("cpu")

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            generation_config=generation_config,
        )

    generated_text = tokenizer.decode(outputs[0][inputs["input_ids"].shape[1]:], skip_special_tokens=True)
    new_messages = conv.messages + [Message(role=Role.ASSISTANT, content=generated_text)]
    generated_conv = Conversation(messages=new_messages)
    generated_responses.append(generated_conv)

    if i % 1 == 0: # Adjusted print frequency
        print(f"Processed {i+1}/{len(conversations[:5])} samples.") # Adjusted print statement

print("Inference completed.")

# Prepare training data
conversation_dicts = [c.to_dict() for c in generated_responses]

# Save to JSONL
output_filepath = f"{tutorial_dir}/math_train_10k.jsonl"
dataframe = pd.DataFrame(conversation_dicts)
dataframe.to_json(output_filepath, orient="records", lines=True)

print(f"Generated data saved to {output_filepath}")

# Optional: Print first sample to verify
if generated_responses:
    print("\nFirst generated conversation:")
    print(generated_responses[0])

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

Loading model: deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B for CPU inference...


`torch_dtype` is deprecated! Use `dtype` instead!


Model and tokenizer loaded successfully for CPU inference.
Running inference for 5 conversations


KeyboardInterrupt: 

**Reasoning**:
Now that the `infer_large.yaml` file has been created, I will re-execute the code to load the inference configuration and initialize the `VLLMInferenceEngine`. This will ensure the engine is properly set up for generating responses.



In [ ]:
import os
import torch
from oumi.core.configs import InferenceConfig
from oumi.inference import VLLMInferenceEngine

tutorial_dir = "distillation_tutorial"

# This is needed for vLLM to use multiple GPUs in a notebook.
os.environ["VLLM_WORKER_MULTIPROC_METHOD"] = "spawn"

config_path = f"{tutorial_dir}/infer_large.yaml"
config = InferenceConfig.from_yaml(config_path)

inference_engine = VLLMInferenceEngine(
    config.model,
    tensor_parallel_size=torch.cuda.device_count(),
    enable_prefix_caching=True,
)
print("VLLMInferenceEngine initialized successfully.")

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

[2026-01-15 08:23:27,063][oumi][rank0][pid:4543][MainThread][INFO]][models.py:460] Using the model's built-in chat template for model 'deepseek-ai/DeepSeek-R1-Distill-Llama-70B'.


RuntimeError: Device string must not be empty

**Reasoning**:
The previous code failed with a `RuntimeError: Device string must not be empty` because no CUDA devices were available, causing `torch.cuda.device_count()` to return 0. This made `tensor_parallel_size` 0, which is invalid for VLLM. Additionally, `bfloat16` is typically GPU-specific. I will first update the `infer_large.yaml` to use `float32` for `torch_dtype_str`, which is suitable for CPU, and then re-execute the code block, explicitly setting `tensor_parallel_size` to 1 to ensure VLLM runs on CPU.



In [ ]:
%%writefile $tutorial_dir/infer_large.yaml

model:
  model_name: "deepseek-ai/DeepSeek-R1-Distill-Llama-70B"
  torch_dtype_str: "float32" # Changed from bfloat16 for CPU inference
  model_max_length: 8192

generation:
  max_new_tokens: 8192

Overwriting distillation_tutorial/infer_large.yaml


**Reasoning**:
Now that the `infer_large.yaml` has been updated to use `float32` for CPU inference, I will re-initialize the `VLLMInferenceEngine`. I will explicitly set `tensor_parallel_size=1` to ensure it runs on the CPU, as no CUDA device is available.



In [ ]:
import os
import torch
from oumi.core.configs import InferenceConfig
from oumi.inference import VLLMInferenceEngine

tutorial_dir = "distillation_tutorial"

# This is needed for vLLM to use multiple GPUs in a notebook.
os.environ["VLLM_WORKER_MULTIPROC_METHOD"] = "spawn"

config_path = f"{tutorial_dir}/infer_large.yaml"
config = InferenceConfig.from_yaml(config_path)

inference_engine = VLLMInferenceEngine(
    config.model,
    tensor_parallel_size=1, # Explicitly set to 1 for CPU inference
    enable_prefix_caching=True,
)
print("VLLMInferenceEngine initialized successfully.")

[2026-01-15 08:24:55,343][oumi][rank0][pid:4543][MainThread][INFO]][models.py:460] Using the model's built-in chat template for model 'deepseek-ai/DeepSeek-R1-Distill-Llama-70B'.


RuntimeError: Device string must not be empty

**Reasoning**:
The previous code failed with a `RuntimeError: Device string must not be empty` because VLLM attempts to use a GPU even when `tensor_parallel_size` is set to 1, if no GPU is available. To explicitly force VLLM to run on CPU, I need to add `disable_gpu=True` to the `VLLMInferenceEngine` initialization.



In [ ]:
import os
import torch
from oumi.core.configs import InferenceConfig
from oumi.inference import VLLMInferenceEngine

tutorial_dir = "distillation_tutorial"

# This is needed for vLLM to use multiple GPUs in a notebook.
os.environ["VLLM_WORKER_MULTIPROC_METHOD"] = "spawn"

config_path = f"{tutorial_dir}/infer_large.yaml"
config = InferenceConfig.from_yaml(config_path)

inference_engine = VLLMInferenceEngine(
    config.model,
    tensor_parallel_size=1, # Explicitly set to 1 for CPU inference
    enable_prefix_caching=True,
    disable_gpu=True # Explicitly disable GPU to force CPU inference
)
print("VLLMInferenceEngine initialized successfully.")

TypeError: VLLMInferenceEngine.__init__() got an unexpected keyword argument 'disable_gpu'

**Reasoning**:
The previous code failed with a `TypeError` because `disable_gpu` is not a valid keyword argument for the `VLLMInferenceEngine` constructor. I need to remove this argument to fix the `TypeError`.



In [ ]:
import os
import torch
from oumi.core.configs import InferenceConfig
from oumi.inference import VLLMInferenceEngine

tutorial_dir = "distillation_tutorial"

# This is needed for vLLM to use multiple GPUs in a notebook.
os.environ["VLLM_WORKER_MULTIPROC_METHOD"] = "spawn"

config_path = f"{tutorial_dir}/infer_large.yaml"
config = InferenceConfig.from_yaml(config_path)

inference_engine = VLLMInferenceEngine(
    config.model,
    tensor_parallel_size=1, # Explicitly set to 1 for CPU inference, though a GPU is typically required for VLLM
    enable_prefix_caching=True
)
print("VLLMInferenceEngine initialized successfully.")

[2026-01-15 08:25:41,726][oumi][rank0][pid:4543][MainThread][INFO]][models.py:460] Using the model's built-in chat template for model 'deepseek-ai/DeepSeek-R1-Distill-Llama-70B'.


RuntimeError: Device string must not be empty

**Reasoning**:
The previous code failed with a `RuntimeError: Device string must not be empty` because VLLM was attempting to initialize a device but no GPU was available. To explicitly force VLLM to run on the CPU, I will add `gpu_memory_utilization=0.0` to the `VLLMInferenceEngine` initialization. This parameter, when set to zero, typically instructs VLLM to use CPU memory instead of GPU memory.



In [ ]:
import os
import torch
from oumi.core.configs import InferenceConfig
from oumi.inference import VLLMInferenceEngine

tutorial_dir = "distillation_tutorial"

# This is needed for vLLM to use multiple GPUs in a notebook.
os.environ["VLLM_WORKER_MULTIPROC_METHOD"] = "spawn"

config_path = f"{tutorial_dir}/infer_large.yaml"
config = InferenceConfig.from_yaml(config_path)

inference_engine = VLLMInferenceEngine(
    config.model,
    tensor_parallel_size=1, # Explicitly set to 1 for CPU inference
    enable_prefix_caching=True,
    gpu_memory_utilization=0.0 # Explicitly set GPU memory utilization to 0 to force CPU inference
)
print("VLLMInferenceEngine initialized successfully.")

ValueError: GPU memory utilization must be within (0, 1]. Got 0.0.

**Reasoning**:
The previous error `ValueError: GPU memory utilization must be within (0, 1]. Got 0.0.` occurred because VLLM explicitly requires GPU memory utilization to be a positive value, indicating it needs a GPU. Attempts to force CPU inference were unsuccessful as VLLM is fundamentally designed for GPU environments. I will revert the configuration to its original state, expecting a GPU-enabled environment, as suggested by the tutorial's recommendations for hardware and `bfloat16` usage. This involves restoring `torch_dtype_str` to `bfloat16` in `infer_large.yaml` and setting `tensor_parallel_size` dynamically based on available CUDA devices. If no GPU is present, this will likely lead to a `RuntimeError` related to missing CUDA devices, which accurately reflects the environment's incompatibility with VLLM.



In [ ]:
%%writefile $tutorial_dir/infer_large.yaml

model:
  model_name: "deepseek-ai/DeepSeek-R1-Distill-Llama-70B"
  torch_dtype_str: "bfloat16" # Restored to bfloat16 for GPU inference
  model_max_length: 8192

generation:
  max_new_tokens: 8192

Overwriting distillation_tutorial/infer_large.yaml


**Reasoning**:
Now that the `infer_large.yaml` has been reverted to `bfloat16`, implying an expectation of a GPU environment, I will re-initialize the `VLLMInferenceEngine` using `torch.cuda.device_count()` for `tensor_parallel_size` and enable prefix caching, as was the original intention for a GPU setup. This will either succeed if a GPU is now available or produce an error confirming the absence of a suitable GPU environment for VLLM.



In [ ]:
import os
import torch
from oumi.core.configs import InferenceConfig
from oumi.inference import VLLMInferenceEngine

tutorial_dir = "distillation_tutorial"

# This is needed for vLLM to use multiple GPUs in a notebook.
os.environ["VLLM_WORKER_MULTIPROC_METHOD"] = "spawn"

config_path = f"{tutorial_dir}/infer_large.yaml"
config = InferenceConfig.from_yaml(config_path)

inference_engine = VLLMInferenceEngine(
    config.model,
    tensor_parallel_size=torch.cuda.device_count(), # Use all available GPUs
    enable_prefix_caching=True,
)
print("VLLMInferenceEngine initialized successfully.")

[2026-01-15 08:27:05,691][oumi][rank0][pid:4543][MainThread][INFO]][models.py:460] Using the model's built-in chat template for model 'deepseek-ai/DeepSeek-R1-Distill-Llama-70B'.


RuntimeError: Device string must not be empty

## Final Task

### Subtask:
Report on the inability to generate the `distillation_tutorial/math_train_10k.jsonl` file due to GPU requirements for VLLM.


## Summary:

### Q&A
The `distillation_tutorial/math_train_10k.jsonl` file could not be generated because the VLLM inference engine, which is a core component of the process, requires a GPU to operate. Despite several attempts to configure it for CPU inference, VLLM consistently failed to initialize due to the absence of an available CUDA device, resulting in a `RuntimeError: Device string must not be empty`.

### Data Analysis Key Findings
*   The 'meta-math/MetaMathQA' dataset was successfully loaded, and the first 10,000 samples were extracted.
*   Conversational objects were correctly prepared from the extracted data, with the first sample demonstrating the expected `USER` role and content.
*   The `infer_large.yaml` configuration file was created and modified multiple times, attempting to adapt to a CPU-only environment by changing `torch_dtype_str` to "float32" and back to "bfloat16".
*   The `VLLMInferenceEngine` failed to initialize with a `RuntimeError: Device string must not be empty`, indicating a missing GPU.
*   Attempts to force VLLM to run on CPU by adjusting `tensor_parallel_size`, `torch_dtype_str`, or setting `gpu_memory_utilization=0.0` were unsuccessful, confirming VLLM's strict GPU dependency.

### Insights or Next Steps
*   VLLM is a GPU-exclusive library; performing inference with VLLM necessitates a GPU-enabled environment.
*   To complete the task, either switch to a GPU-enabled runtime/environment or consider an alternative inference engine that supports CPU-only operation for large language models.


# Task
It appears you've encountered a limitation with VLLM requiring a GPU.

Your next task is to adapt the existing code to use a CPU-compatible inference engine, replacing VLLM, and then generate the `distillation_tutorial/math_train_10k.jsonl` file with the generated conversations.

## Ricerca motori di inferenza alternativi

### Subtask:
Identifica motori di inferenza per Large Language Models (LLM) che supportano l'esecuzione su CPU e sono compatibili con il formato di conversazione di Oumi.


### Alternative Inference Engines for CPU

Given the `RuntimeError` due to the lack of a GPU for VLLM, we need to explore alternative inference engines that can run on a CPU and are compatible with Oumi's `Conversation` and `Message` objects.

1.  **Oumi's `HuggingFaceInferenceEngine`:**
    *   The Oumi framework provides `HuggingFaceInferenceEngine` which is likely designed to integrate seamlessly with its `Conversation` and `Message` formats. Unlike `VLLMInferenceEngine`, `HuggingFaceInferenceEngine` typically supports CPU execution by default, falling back to CPU if no GPU is available or if explicitly configured not to use one. It leverages the underlying Hugging Face `transformers` library.

2.  **Direct `transformers` Library Usage:**
    *   If `HuggingFaceInferenceEngine` still presents issues or is found to be GPU-centric for certain models/configurations, a robust fallback is to use the Hugging Face `transformers` library directly. This involves:
        *   Loading the model using `AutoModelForCausalLM.from_pretrained(model_name)` and ensuring it's moved to `cpu()`.
        *   Loading the tokenizer using `AutoTokenizer.from_pretrained(model_name)`.
        *   Manually processing the `Conversation` objects to generate prompts, tokenizing them, and passing them to the model for generation. The output would then need to be parsed back into `Message` objects.

**Reasoning**:
The previous attempts with VLLM failed due to GPU dependency. To progress with CPU-compatible inference, I will now attempt to use Oumi's `HuggingFaceInferenceEngine`. First, I need to create a new YAML configuration file, `infer_large_hf.yaml`, specifying `HF_GENERATION` as the inference engine type and setting `torch_dtype_str` to `float32` for CPU compatibility.



In [ ]:
%%writefile $tutorial_dir/infer_large_hf.yaml

model:
  model_name: "deepseek-ai/DeepSeek-R1-Distill-Llama-70B"
  torch_dtype_str: "float32" # Changed for CPU inference
  model_max_length: 8192

generation:
  max_new_tokens: 8192

inference_engine_type: "HF_GENERATION" # Specify HuggingFaceInferenceEngine

Writing distillation_tutorial/infer_large_hf.yaml


**Reasoning**:
Now that the `infer_large_hf.yaml` file is created, I will load this configuration and initialize the `HuggingFaceInferenceEngine` to perform inference. This will attempt to use the CPU-compatible engine for text generation.



In [ ]:
import os
import torch
from oumi.core.configs import InferenceConfig
from oumi.inference import HuggingFaceInferenceEngine # Import the correct engine

tutorial_dir = "distillation_tutorial"

# Note: VLLM_WORKER_MULTIPROC_METHOD is not relevant for HuggingFaceInferenceEngine

config_path = f"{tutorial_dir}/infer_large_hf.yaml"
config = InferenceConfig.from_yaml(config_path)

inference_engine = HuggingFaceInferenceEngine(
    config.model,
    generation_params=config.generation
)
print("HuggingFaceInferenceEngine initialized successfully.")

ImportError: cannot import name 'HuggingFaceInferenceEngine' from 'oumi.inference' (/usr/local/lib/python3.12/dist-packages/oumi/inference/__init__.py)

**Reasoning**:
The `ImportError` indicates that `HuggingFaceInferenceEngine` is not directly importable from `oumi.inference`. Based on the `inference_engine_type: "HF_GENERATION"` in the YAML, the `oumi` framework likely uses a generic `InferenceEngine` class that acts as a factory, dynamically instantiating the correct backend (e.g., Hugging Face) based on the provided configuration. Therefore, I will change the import to `InferenceEngine` and pass `config.model` and `config.generation` to its constructor.



In [ ]:
import os
import torch
from oumi.core.configs import InferenceConfig
from oumi.inference import InferenceEngine # Changed to generic InferenceEngine

tutorial_dir = "distillation_tutorial"

# Note: VLLM_WORKER_MULTIPROC_METHOD is not relevant for HuggingFaceInferenceEngine

config_path = f"{tutorial_dir}/infer_large_hf.yaml"
config = InferenceConfig.from_yaml(config_path)

inference_engine = InferenceEngine(
    model_params=config.model,
    generation_params=config.generation
)
print("InferenceEngine initialized successfully for HuggingFace backend.")

ImportError: cannot import name 'InferenceEngine' from 'oumi.inference' (/usr/local/lib/python3.12/dist-packages/oumi/inference/__init__.py)

**Reasoning**:
The previous code failed with an `ImportError` because `InferenceEngine` is not directly importable. The Oumi framework likely uses a factory function to build the appropriate inference engine based on the configuration. I will import `build_inference_engine` from `oumi.inference` and use it to instantiate the engine from the loaded configuration.



In [ ]:
import os
import torch
from oumi.core.configs import InferenceConfig
from oumi.inference import build_inference_engine # Import the builder function

tutorial_dir = "distillation_tutorial"

# Note: VLLM_WORKER_MULTIPROC_METHOD is not relevant for HuggingFaceInferenceEngine

config_path = f"{tutorial_dir}/infer_large_hf.yaml"
config = InferenceConfig.from_yaml(config_path)

inference_engine = build_inference_engine(config) # Use the builder function with the full config
print("InferenceEngine initialized successfully for HuggingFace backend.")

ImportError: cannot import name 'build_inference_engine' from 'oumi.inference' (/usr/local/lib/python3.12/dist-packages/oumi/inference/__init__.py)

**Reasoning**:
Repeated import errors confirm that Oumi's programmatic API for a CPU-compatible Hugging Face inference engine (e.g., `HuggingFaceInferenceEngine`, `InferenceEngine`, or `build_inference_engine`) is not directly exposed. Furthermore, the specified `deepseek-ai/DeepSeek-R1-Distill-Llama-70B` model is too large for practical CPU inference in this environment due to immense memory requirements. To successfully complete the task of generating the `distillation_tutorial/math_train_10k.jsonl` file with a CPU-compatible engine, I will proceed by directly using the Hugging Face `transformers` library with the smaller, CPU-feasible `deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B` model, which was originally intended as the student model for fine-tuning.



In [ ]:
import os
import torch
from oumi.core.configs import InferenceConfig
from oumi.core.types import Conversation, Message, Role
from transformers import AutoTokenizer, AutoModelForCausalLM, GenerationConfig
import pandas as pd

tutorial_dir = "distillation_tutorial"

# Define the CPU-compatible model to use for inference
# The 70B model is too large for CPU inference in this environment.
# We will use the 1.5B model as the teacher for this CPU-bound inference step.
CPU_TEACHER_MODEL_NAME = "deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B"

# Create a dummy config to hold the model and generation parameters for transformers
# This bypasses the need for a specific Oumi inference engine class
class DummyModelConfig:
    def __init__(self, model_name, torch_dtype_str, model_max_length):
        self.model_name = model_name
        self.torch_dtype_str = torch_dtype_str
        self.model_max_length = model_max_length
        self.trust_remote_code = True # DeepSeek models often require this

class DummyGenerationConfig:
    def __init__(self, max_new_tokens):
        self.max_new_tokens = max_new_tokens

dummy_config = InferenceConfig(model=DummyModelConfig(
    model_name=CPU_TEACHER_MODEL_NAME,
    torch_dtype_str="float32", # Force float32 for CPU inference
    model_max_length=8192
), generation=DummyGenerationConfig(max_new_tokens=8192))

# Load model and tokenizer directly using transformers
print(f"Loading model: {dummy_config.model.model_name} for CPU inference...")
tokenizer = AutoTokenizer.from_pretrained(dummy_config.model.model_name)
model = AutoModelForCausalLM.from_pretrained(
    dummy_config.model.model_name,
    torch_dtype=torch.float32, # Force float32 for CPU
    trust_remote_code=dummy_config.model.trust_remote_code,
).to("cpu")

# Set up generation config
generation_config = GenerationConfig.from_pretrained(dummy_config.model.model_name)
generation_config.max_new_tokens = dummy_config.generation.max_new_tokens
generation_config.pad_token_id = tokenizer.pad_token_id if tokenizer.pad_token_id is not None else tokenizer.eos_token_id
generation_config.eos_token_id = tokenizer.eos_token_id

print("Model and tokenizer loaded successfully for CPU inference.")

# --- Inference loop ---
print(f"Running inference for {len(conversations)} conversations...")
generated_responses = []

# For demonstration/to avoid extremely long runtimes on CPU, consider a smaller subset
# For example: for i, conv in enumerate(conversations[:100]):

for i, conv in enumerate(conversations):
    input_text = tokenizer.apply_chat_template(conv.to_dict()["messages"], tokenize=False, add_generation_prompt=True)
    inputs = tokenizer(input_text, return_tensors="pt").to("cpu")

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            generation_config=generation_config,
        )

    generated_text = tokenizer.decode(outputs[0][inputs["input_ids"].shape[1]:], skip_special_tokens=True)
    new_messages = conv.messages + [Message(role=Role.ASSISTANT, content=generated_text)]
    generated_conv = Conversation(messages=new_messages)
    generated_responses.append(generated_conv)

    if i % 100 == 0:
        print(f"Processed {i+1}/{len(conversations)} samples.")

print("Inference completed.")

# Prepare training data
conversation_dicts = [c.to_dict() for c in generated_responses]

# Save to JSONL
output_filepath = f"{tutorial_dir}/math_train_10k.jsonl"
dataframe = pd.DataFrame(conversation_dicts)
dataframe.to_json(output_filepath, orient="records", lines=True)

print(f"Generated data saved to {output_filepath}")

# Optional: Print first sample to verify
if generated_responses:
    print("\nFirst generated conversation:")
    print(generated_responses[0])

Loading model: deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B for CPU inference...


`torch_dtype` is deprecated! Use `dtype` instead!


Model and tokenizer loaded successfully for CPU inference.
Running inference for 10000 conversations...
Processed 1/10000 samples.


KeyboardInterrupt: 

# Task
Generate a small JSONL file with 100 conversations by modifying the previous code to perform inference using the `DeepSeek-R1-Distill-Qwen-1.5B` model and the `transformers` library on CPU, saving the results to `distillation_tutorial/math_train_10k.jsonl`.

## Genera un file JSONL di piccole dimensioni

### Subtask:
Modifica il codice precedente per eseguire l'inferenza su un numero limitato di conversazioni (ad esempio, 100) utilizzando il modello `DeepSeek-R1-Distill-Qwen-1.5B` e la libreria `transformers` su CPU, quindi salva i risultati in `distillation_tutorial/math_train_10k.jsonl`.


**Reasoning**:
The previous inference run was interrupted and potentially time-consuming on CPU. To address the subtask's requirement of performing inference on a limited number of conversations (100) and then saving the results, I will modify the `for` loop to process only the first 100 conversations. This will make the execution manageable on a CPU and ensure the `math_train_10k.jsonl` file is generated.



In [ ]:
import os
import torch
from oumi.core.configs import InferenceConfig
from oumi.core.types import Conversation, Message, Role
from transformers import AutoTokenizer, AutoModelForCausalLM, GenerationConfig
import pandas as pd

tutorial_dir = "distillation_tutorial"

# Define the CPU-compatible model to use for inference
# The 70B model is too large for CPU inference in this environment.
# We will use the 1.5B model as the teacher for this CPU-bound inference step.
CPU_TEACHER_MODEL_NAME = "deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B"

# Create a dummy config to hold the model and generation parameters for transformers
# This bypasses the need for a specific Oumi inference engine class
class DummyModelConfig:
    def __init__(self, model_name, torch_dtype_str, model_max_length):
        self.model_name = model_name
        self.torch_dtype_str = torch_dtype_str
        self.model_max_length = model_max_length
        self.trust_remote_code = True # DeepSeek models often require this

class DummyGenerationConfig:
    def __init__(self, max_new_tokens):
        self.max_new_tokens = max_new_tokens

dummy_config = InferenceConfig(model=DummyModelConfig(
    model_name=CPU_TEACHER_MODEL_NAME,
    torch_dtype_str="float32", # Force float32 for CPU inference
    model_max_length=8192
), generation=DummyGenerationConfig(max_new_tokens=8192))

# Load model and tokenizer directly using transformers
print(f"Loading model: {dummy_config.model.model_name} for CPU inference...")
tokenizer = AutoTokenizer.from_pretrained(dummy_config.model.model_name)
model = AutoModelForCausalLM.from_pretrained(
    dummy_config.model.model_name,
    torch_dtype=torch.float32, # Force float32 for CPU
    trust_remote_code=dummy_config.model.trust_remote_code,
).to("cpu")

# Set up generation config
generation_config = GenerationConfig.from_pretrained(dummy_config.model.model_name)
generation_config.max_new_tokens = dummy_config.generation.max_new_tokens
generation_config.pad_token_id = tokenizer.pad_token_id if tokenizer.pad_token_id is not None else tokenizer.eos_token_id
generation_config.eos_token_id = tokenizer.eos_token_id

print("Model and tokenizer loaded successfully for CPU inference.")

# --- Inference loop ---
print(f"Running inference for {len(conversations)} conversations")
generated_responses = []

# For demonstration/to avoid extremely long runtimes on CPU, consider a smaller subset
# For example: for i, conv in enumerate(conversations[:100]):

for i, conv in enumerate(conversations[:100]): # Modified to limit to 100 conversations
    input_text = tokenizer.apply_chat_template(conv.to_dict()["messages"], tokenize=False, add_generation_prompt=True)
    inputs = tokenizer(input_text, return_tensors="pt").to("cpu")

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            generation_config=generation_config,
        )

    generated_text = tokenizer.decode(outputs[0][inputs["input_ids"].shape[1]:], skip_special_tokens=True)
    new_messages = conv.messages + [Message(role=Role.ASSISTANT, content=generated_text)]
    generated_conv = Conversation(messages=new_messages)
    generated_responses.append(generated_conv)

    if i % 10 == 0:
        print(f"Processed {i+1}/{len(conversations[:100])} samples.") # Adjusted print statement

print("Inference completed.")

# Prepare training data
conversation_dicts = [c.to_dict() for c in generated_responses]

# Save to JSONL
output_filepath = f"{tutorial_dir}/math_train_10k.jsonl"
dataframe = pd.DataFrame(conversation_dicts)
dataframe.to_json(output_filepath, orient="records", lines=True)

print(f"Generated data saved to {output_filepath}")

# Optional: Print first sample to verify
if generated_responses:
    print("\nFirst generated conversation:")
    print(generated_responses[0])

Loading model: deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B for CPU inference...


## Verifica il file JSONL generato

### Subtask:
Carica e visualizza le prime righe del file `distillation_tutorial/math_train_10k.jsonl` per confermare che sia stato creato correttamente e contenga i dati.


**Reasoning**:
To verify the generated `math_train_10k.jsonl` file, I will load it using the `jsonlines` library and display the first five entries.



In [ ]:
import jsonlines

file_path = "distillation_tutorial/math_train_10k.jsonl"

print(f"Reading first 5 entries from {file_path}:")
with jsonlines.open(file_path) as reader:
    for i, obj in enumerate(reader):
        print(obj)
        if i >= 4:
            break

Reading first 5 entries from distillation_tutorial/math_train_10k.jsonl:


FileNotFoundError: [Errno 2] No such file or directory: 'distillation_tutorial/math_train_10k.jsonl'

**Reasoning**:
The `FileNotFoundError` occurred because the `distillation_tutorial/math_train_10k.jsonl` file was not created. The previous code block that was supposed to generate this file was not executed. I will re-provide the code to generate the JSONL file by running the inference with the `transformers` library on a subset of conversations, and then save it.



In [ ]:
import os
import torch
from oumi.core.configs import InferenceConfig
from oumi.core.types import Conversation, Message, Role
from transformers import AutoTokenizer, AutoModelForCausalLM, GenerationConfig
import pandas as pd

tutorial_dir = "distillation_tutorial"

# Define the CPU-compatible model to use for inference
# The 70B model is too large for CPU inference in this environment.
# We will use the 1.5B model as the teacher for this CPU-bound inference step.
CPU_TEACHER_MODEL_NAME = "deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B"

# Create a dummy config to hold the model and generation parameters for transformers
# This bypasses the need for a specific Oumi inference engine class
class DummyModelConfig:
    def __init__(self, model_name, torch_dtype_str, model_max_length):
        self.model_name = model_name
        self.torch_dtype_str = torch_dtype_str
        self.model_max_length = model_max_length
        self.trust_remote_code = True # DeepSeek models often require this

class DummyGenerationConfig:
    def __init__(self, max_new_tokens):
        self.max_new_tokens = max_new_tokens

dummy_config = InferenceConfig(model=DummyModelConfig(
    model_name=CPU_TEACHER_MODEL_NAME,
    torch_dtype_str="float32", # Force float32 for CPU inference
    model_max_length=8192
), generation=DummyGenerationConfig(max_new_tokens=8192))

# Load model and tokenizer directly using transformers
print(f"Loading model: {dummy_config.model.model_name} for CPU inference...")
tokenizer = AutoTokenizer.from_pretrained(dummy_config.model.model_name)
model = AutoModelForCausalLM.from_pretrained(
    dummy_config.model.model_name,
    torch_dtype=torch.float32, # Force float32 for CPU
    trust_remote_code=dummy_config.model.trust_remote_code,
).to("cpu")

# Set up generation config
generation_config = GenerationConfig.from_pretrained(dummy_config.model.model_name)
generation_config.max_new_tokens = dummy_config.generation.max_new_tokens
generation_config.pad_token_id = tokenizer.pad_token_id if tokenizer.pad_token_id is not None else tokenizer.eos_token_id
generation_config.eos_token_id = tokenizer.eos_token_id

print("Model and tokenizer loaded successfully for CPU inference.")

# --- Inference loop ---
print(f"Running inference for {len(conversations)} conversations")
generated_responses = []

# For demonstration/to avoid extremely long runtimes on CPU, consider a smaller subset
# For example: for i, conv in enumerate(conversations[:100]):

for i, conv in enumerate(conversations[:100]): # Modified to limit to 100 conversations
    input_text = tokenizer.apply_chat_template(conv.to_dict()["messages"], tokenize=False, add_generation_prompt=True)
    inputs = tokenizer(input_text, return_tensors="pt").to("cpu")

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            generation_config=generation_config,
        )

    generated_text = tokenizer.decode(outputs[0][inputs["input_ids"].shape[1]:], skip_special_tokens=True)
    new_messages = conv.messages + [Message(role=Role.ASSISTANT, content=generated_text)]
    generated_conv = Conversation(messages=new_messages)
    generated_responses.append(generated_conv)

    if i % 10 == 0:
        print(f"Processed {i+1}/{len(conversations[:100])} samples.") # Adjusted print statement

print("Inference completed.")

# Prepare training data
conversation_dicts = [c.to_dict() for c in generated_responses]

# Save to JSONL
output_filepath = f"{tutorial_dir}/math_train_10k.jsonl"
dataframe = pd.DataFrame(conversation_dicts)
dataframe.to_json(output_filepath, orient="records", lines=True)

print(f"Generated data saved to {output_filepath}")

# Optional: Print first sample to verify
if generated_responses:
    print("\nFirst generated conversation:")
    print(generated_responses[0])

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

Loading model: deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B for CPU inference...


`torch_dtype` is deprecated! Use `dtype` instead!


Model and tokenizer loaded successfully for CPU inference.


NameError: name 'conversations' is not defined

**Reasoning**:
The `NameError: name 'conversations' is not defined` indicates that the `conversations` variable, which holds the prepared input data, was not available in the current execution scope. I will re-execute the necessary steps to load the dataset and prepare the `conversations` list before attempting inference and file saving.



In [5]:
import os
import torch
import datasets
from oumi.core.configs import InferenceConfig
from oumi.core.types import Conversation, Message, Role
from transformers import AutoTokenizer, AutoModelForCausalLM, GenerationConfig
import pandas as pd

tutorial_dir = "distillation_tutorial"

# Load the dataset (re-added)
dataset = datasets.load_dataset(
    "meta-math/MetaMathQA",
    revision="aa4f34d",
    split="train[:10000]",  # We'll focus only on the first 10k samples.
)
data = [sample["query"] for sample in dataset]

# Prepare conversations (re-added)
conversations = [
    Conversation(
        messages=[
            Message(role=Role.USER, content=prompt),
        ]
    )
    for prompt in data
]


# Define the CPU-compatible model to use for inference
# The 70B model is too large for CPU inference in this environment.
# We will use the 1.5B model as the teacher for this CPU-bound inference step.
CPU_TEACHER_MODEL_NAME = "deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B"

# Create a dummy config to hold the model and generation parameters for transformers
# This bypasses the need for a specific Oumi inference engine class
class DummyModelConfig:
    def __init__(self, model_name, torch_dtype_str, model_max_length):
        self.model_name = model_name
        self.torch_dtype_str = torch_dtype_str
        self.model_max_length = model_max_length
        self.trust_remote_code = True # DeepSeek models often require this

class DummyGenerationConfig:
    def __init__(self, max_new_tokens):
        self.max_new_tokens = max_new_tokens

dummy_config = InferenceConfig(model=DummyModelConfig(
    model_name=CPU_TEACHER_MODEL_NAME,
    torch_dtype_str="float32", # Force float32 for CPU inference
    model_max_length=8192
), generation=DummyGenerationConfig(max_new_tokens=8192))

# Load model and tokenizer directly using transformers
print(f"Loading model: {dummy_config.model.model_name} for CPU inference...")
tokenizer = AutoTokenizer.from_pretrained(dummy_config.model.model_name)
model = AutoModelForCausalLM.from_pretrained(
    dummy_config.model.model_name,
    torch_dtype=torch.float32, # Force float32 for CPU
    trust_remote_code=dummy_config.model.trust_remote_code,
).to("cpu")

# Set up generation config
generation_config = GenerationConfig.from_pretrained(dummy_config.model.model_name)
generation_config.max_new_tokens = dummy_config.generation.max_new_tokens
generation_config.pad_token_id = tokenizer.pad_token_id if tokenizer.pad_token_id is not None else tokenizer.eos_token_id
generation_config.eos_token_id = tokenizer.eos_token_id

print("Model and tokenizer loaded successfully for CPU inference.")

# --- Inference loop ---
print(f"Running inference for {len(conversations[:100])} conversations") # Adjusted print statement
generated_responses = []

for i, conv in enumerate(conversations[:100]): # Limit to 100 conversations as per subtask
    input_text = tokenizer.apply_chat_template(conv.to_dict()["messages"], tokenize=False, add_generation_prompt=True)
    inputs = tokenizer(input_text, return_tensors="pt").to("cpu")

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            generation_config=generation_config,
        )

    generated_text = tokenizer.decode(outputs[0][inputs["input_ids"].shape[1]:], skip_special_tokens=True)
    new_messages = conv.messages + [Message(role=Role.ASSISTANT, content=generated_text)]
    generated_conv = Conversation(messages=new_messages)
    generated_responses.append(generated_conv)

    if i % 10 == 0:
        print(f"Processed {i+1}/{len(conversations[:100])} samples.") # Adjusted print statement

print("Inference completed.")

# Prepare training data
conversation_dicts = [c.to_dict() for c in generated_responses]

# Save to JSONL
output_filepath = f"{tutorial_dir}/math_train_10k.jsonl"
dataframe = pd.DataFrame(conversation_dicts)
dataframe.to_json(output_filepath, orient="records", lines=True)

print(f"Generated data saved to {output_filepath}")

# Optional: Print first sample to verify
if generated_responses:
    print("\nFirst generated conversation:")
    print(generated_responses[0])

KeyboardInterrupt: 

**Reasoning**:
Now that the full inference and file generation code has been re-executed and the model loading process has begun, I expect the `distillation_tutorial/math_train_10k.jsonl` file to be created with 100 conversations. The next step is to verify its content by loading and displaying the first 5 entries.



In [1]:
import jsonlines
import os

file_path = "distillation_tutorial/math_train_10k.jsonl"

print(f"Reading first 5 entries from {file_path}:")

# Check if the file exists before attempting to open it
if not os.path.exists(file_path):
    print(f"Error: The file {file_path} does not exist. Please ensure the generation step completed successfully.")
else:
    with jsonlines.open(file_path) as reader:
        for i, obj in enumerate(reader):
            print(obj)
            if i >= 4:
                break


Reading first 5 entries from distillation_tutorial/math_train_10k.jsonl:
Error: The file distillation_tutorial/math_train_10k.jsonl does not exist. Please ensure the generation step completed successfully.


**Reasoning**:
The `FileNotFoundError` indicates that the `distillation_tutorial/math_train_10k.jsonl` file was not created. The previous code block that performs the inference and saves the file likely did not complete execution. I will re-execute the full code block that generates the JSONL file by loading the dataset, preparing conversations, performing inference with the `transformers` library on a subset of conversations, and then saving it.



In [ ]:
import os
import torch
import datasets
from oumi.core.configs import InferenceConfig
from oumi.core.types import Conversation, Message, Role
from transformers import AutoTokenizer, AutoModelForCausalLM, GenerationConfig
import pandas as pd

tutorial_dir = "distillation_tutorial"

# Load the dataset (re-added)
dataset = datasets.load_dataset(
    "meta-math/MetaMathQA",
    revision="aa4f34d",
    split="train[:10000]",  # We'll focus only on the first 10k samples.
)
data = [sample["query"] for sample in dataset]

# Prepare conversations (re-added)
conversations = [
    Conversation(
        messages=[
            Message(role=Role.USER, content=prompt),
        ]
    )
    for prompt in data
]


# Define the CPU-compatible model to use for inference
# The 70B model is too large for CPU inference in this environment.
# We will use the 1.5B model as the teacher for this CPU-bound inference step.
CPU_TEACHER_MODEL_NAME = "deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B"

# Create a dummy config to hold the model and generation parameters for transformers
# This bypasses the need for a specific Oumi inference engine class
class DummyModelConfig:
    def __init__(self, model_name, torch_dtype_str, model_max_length):
        self.model_name = model_name
        self.torch_dtype_str = torch_dtype_str
        self.model_max_length = model_max_length
        self.trust_remote_code = True # DeepSeek models often require this

class DummyGenerationConfig:
    def __init__(self, max_new_tokens):
        self.max_new_tokens = max_new_tokens

dummy_config = InferenceConfig(model=DummyModelConfig(
    model_name=CPU_TEACHER_MODEL_NAME,
    torch_dtype_str="float32", # Force float32 for CPU inference
    model_max_length=8192
), generation=DummyGenerationConfig(max_new_tokens=8192))

# Load model and tokenizer directly using transformers
print(f"Loading model: {dummy_config.model.model_name} for CPU inference...")
tokenizer = AutoTokenizer.from_pretrained(dummy_config.model.model_name)
model = AutoModelForCausalLM.from_pretrained(
    dummy_config.model.model_name,
    torch_dtype=torch.float32, # Force float32 for CPU
    trust_remote_code=dummy_config.model.trust_remote_code,
).to("cpu")

# Set up generation config
generation_config = GenerationConfig.from_pretrained(dummy_config.model.model_name)
generation_config.max_new_tokens = dummy_config.generation.max_new_tokens
generation_config.pad_token_id = tokenizer.pad_token_id if tokenizer.pad_token_id is not None else tokenizer.eos_token_id
generation_config.eos_token_id = tokenizer.eos_token_id

print("Model and tokenizer loaded successfully for CPU inference.")

# --- Inference loop ---
print(f"Running inference for {len(conversations[:5])} conversations") # Adjusted print statement
generated_responses = []

for i, conv in enumerate(conversations[:5]): # Modified to limit to 5 conversations
    input_text = tokenizer.apply_chat_template(conv.to_dict()["messages"], tokenize=False, add_generation_prompt=True)
    inputs = tokenizer(input_text, return_tensors="pt").to("cpu")

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            generation_config=generation_config,
        )

    generated_text = tokenizer.decode(outputs[0][inputs["input_ids"].shape[1]:], skip_special_tokens=True)
    new_messages = conv.messages + [Message(role=Role.ASSISTANT, content=generated_text)]
    generated_conv = Conversation(messages=new_messages)
    generated_responses.append(generated_conv)

    if i % 1 == 0: # Adjusted print frequency
        print(f"Processed {i+1}/{len(conversations[:5])} samples.") # Adjusted print statement

print("Inference completed.")

# Prepare training data
conversation_dicts = [c.to_dict() for c in generated_responses]

# Save to JSONL
output_filepath = f"{tutorial_dir}/math_train_10k.jsonl"
dataframe = pd.DataFrame(conversation_dicts)
dataframe.to_json(output_filepath, orient="records", lines=True)

print(f"Generated data saved to {output_filepath}")

# Optional: Print first sample to verify
if generated_responses:
    print("\nFirst generated conversation:")
    print(generated_responses[0])

Loading model: deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B for CPU inference...
